In [ ]:

import requests

url = 'https://api.trello.com/1/webhooks/'
params = {
    'callbackURL': 'sqlhelper.com.br/webhook',  # Substitua pela sua URL de callback
    'idModel': '645d11919349f344ad96d4ba',  # Substitua pelo ID do modelo que deseja monitorar
    'key': '82fbbfa646064873d7f742d12c2b3f28',  # Substitua pela sua chave de API do Trello
    'token': 'ATTA96d4de928b043159b73889c1bfe1c1d716e6ca7a03d558a966b4ec97e4bc8c3e55BD66EB',  # Substitua pelo seu token de API do Trello
}

try:
    response = requests.post(url, params=params)
    response.raise_for_status()  # Lança uma exceção se a resposta contém um código de status de erro

    if response.status_code == 200:
        print("Webhook criado com sucesso!")
    else:
        print("Falha ao criar o webhook. Código de status:", response.status_code)

except requests.exceptions.RequestException as e:
    print("Ocorreu um erro durante a solicitação:", str(e))


Ocorreu um erro durante a solicitação: 400 Client Error: Bad Request for url: https://api.trello.com/1/webhooks/?callbackURL=https%3A%2F%2Fbd08-191-162-253-229.ngrok-free.app%2Fwebhook&idModel=645d11919349f344ad96d4ba&key=82fbbfa646064873d7f742d12c2b3f28&token=ATTA96d4de928b043159b73889c1bfe1c1d716e6ca7a03d558a966b4ec97e4bc8c3e55BD66EB


In [ ]:
import requests
import pymysql

url = "https://api.tiny.com.br/api2/produtos.pesquisa.php"
token = "bc3cdea243d8687963fa642580057531456d34fa"
produtos_totais = []
registros_inseridos = 0

for pagina in range(1, 38):
    params = {
        "token": token,
        "formato": "json",
        "pesquisa": "",
        "pagina": pagina
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Processar a resposta (data) conforme necessário
    # ...

    produtos_pagina = data['retorno']['produtos']
    produtos_totais.extend(produtos_pagina)

# Conectar ao banco de dados MySQL usando PyMySQL
conn = pymysql.connect(
    host='mysql.sqlhelper.com.br',
    user='sqlhelper01',
    password='140477nagy',
    database='sqlhelper01'
)
cursor = conn.cursor()

# Executar as instruções SQL no banco de dados e contar os registros inseridos
for produto in produtos_totais:
    produto_info = produto['produto']
    sql_insert = "INSERT INTO produtos (id, data_criacao, nome, codigo, preco, preco_promocional, unidade, gtin, tipoVariacao, localizacao, preco_custo, preco_custo_medio, situacao) "
    sql_insert += "VALUES ('" + produto_info['id'] + "', STR_TO_DATE('" + produto_info['data_criacao'] + "', '%d/%m/%Y %H:%i:%s'), "
    sql_insert += "'" + produto_info['nome'] + "', '" + produto_info['codigo'] + "', " + str(produto_info['preco']) + ", " + str(produto_info['preco_promocional']) + ", "
    sql_insert += "'" + produto_info['unidade'] + "', '" + produto_info['gtin'] + "', '" + produto_info['tipoVariacao'] + "', '" + produto_info['localizacao'] + "', "
    sql_insert += str(produto_info['preco_custo']) + ", " + str(produto_info['preco_custo_medio']) + ", '" + produto_info['situacao'] + "');"

    try:
        cursor.execute(sql_insert)
        registros_inseridos += 1
    except pymysql.Error as e:
        print(f"Erro ao inserir registro: {e}")

# Confirmar as alterações no banco de dados
conn.commit()

# Fechar a conexão com o banco de dados
cursor.close()
conn.close()

# Exibir o número de registros inseridos
print(f"Total de registros inseridos: {registros_inseridos}")


Total de registros inseridos: 3670


In [ ]:

for produto in produtos_totais:
    produto_info = produto['produto']
    linha = f"ID: {produto_info['id']}, Nome: {produto_info['nome']}, Código: {produto_info['codigo']}, Preço: {produto_info['preco']}, Situação: {produto_info['situacao']}"
    print(linha)
print(f"Total de linhas: {contador}")


In [13]:
import pandas as pd
from sqlalchemy import create_engine

# Read the Excel file into a pandas DataFrame
planilha = pd.read_excel('C:\\Users\\Administrator\\Documents\\GitHub\\API-Arte-arena\\arquivos\\Book1.xlsx')

# Define the database connection parameters
endereco_banco_dados = 'mysql.sqlhelper.com.br'
usuario = 'sqlhelper01'
senha = '140477nagy'
nome_banco_dados = 'sqlhelper01'

# Create the SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{endereco_banco_dados}/{nome_banco_dados}')

# Define the table name to insert data
nome_tabela = 'pedidos'

# Assuming the 'id' column is named 'id' in your DataFrame
id_column_name = 'id'

# Fetching the existing ids from the database to check for duplicates
existing_ids_query = f"SELECT {id_column_name} FROM {nome_tabela};"
existing_ids = pd.read_sql_query(existing_ids_query, con=engine)

# Filtering out the rows that have existing ids in the DataFrame
new_data = planilha[~planilha[id_column_name].isin(existing_ids[id_column_name])]

# Remove duplicates from the DataFrame based on the 'id' column
new_data.drop_duplicates(subset=id_column_name, keep='first', inplace=True)

# Inserting the new data into the database
if not new_data.empty:
    new_data.to_sql(nome_tabela, con=engine, if_exists='append', index=False)

print("Data inserted successfully.")


Data inserted successfully.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7560\356040108.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop_duplicates(subset=id_column_name, keep='first', inplace=True)
